In [1]:
#!pip install opencv-python

In [2]:
# TRANSFER LEARNING FINE TUNING -  MASS CLASSIFICATION USING SEGMENTATION MODEL OUTPUT BCDR MASS LESION
import numpy as np
import keras
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
#%matplotlib inline
keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
#import pandas as pd
import os
#import matplotlib.pyplot as plt
#from tensorflow.python.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU, Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras import Sequential, Model

vid='v003_5' #last number is the cross-validation fold number

In [3]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications import InceptionV3
base_model=InceptionV3(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(128)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 25, 25, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 25, 25, 32)  96          ['conv2d_11[0][0]']              
 ormalizat

                                                                                                  
 batch_normalization_22 (BatchN  (None, 25, 25, 64)  192         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_20 (Conv2D)             (None, 25, 25, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_23 (Conv2D)             (None, 25, 25, 96)   55296       ['activation_22[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 12, 12, 288)  0          ['mixed2[0][0]']                 
                                                                                                  
 mixed3 (Concatenate)           (None, 12, 12, 768)  0           ['activation_26[0][0]',          
                                                                  'activation_29[0][0]',          
                                                                  'max_pooling2d_2[0][0]']        
                                                                                                  
 conv2d_34 (Conv2D)             (None, 12, 12, 128)  98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_34 (BatchN  (None, 12, 12, 128)  384        ['conv2d_34[0][0]']              
 ormalizat

                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_44 (Conv2D)             (None, 12, 12, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_normalization_44 (BatchN  (None, 12, 12, 160)  480        ['conv2d_44[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_44 (Activation)     (None, 12, 12, 160)  0           ['batch_normalization_44[0][0]'] 
                                                                                                  
 conv2d_45 (Conv2D)             (None, 12, 12, 160)  179200      ['activation_44[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 12, 12, 160)  0           ['batch_normalization_54[0][0]'] 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 12, 12, 160)  179200      ['activation_54[0][0]']          
                                                                                                  
 batch_normalization_55 (BatchN  (None, 12, 12, 160)  480        ['conv2d_55[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_55 (Activation)     (None, 12, 12, 160)  0           ['batch_normalization_55[0][0]'] 
          

                                                                                                  
 batch_normalization_65 (BatchN  (None, 12, 12, 192)  576        ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_65 (Activation)     (None, 12, 12, 192)  0           ['batch_normalization_65[0][0]'] 
                                                                                                  
 conv2d_61 (Conv2D)             (None, 12, 12, 192)  147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_66 (Conv2D)             (None, 12, 12, 192)  258048      ['activation_65[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_70 (Conv2D)             (None, 12, 12, 192)  147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_74 (Conv2D)             (None, 12, 12, 192)  258048      ['activation_73[0][0]']          
                                                                                                  
 batch_normalization_70 (BatchN  (None, 12, 12, 192)  576        ['conv2d_70[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_74 (BatchN  (None, 12, 12, 192)  576        ['conv2d_74[0][0]']              
 ormalization)                                                                                    
          

 batch_normalization_76 (BatchN  (None, 5, 5, 320)   960         ['conv2d_76[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_78 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_78[0][0]'] 
                                                                                                  
 activation_79 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_79[0][0]'] 
                                                                                                  
 activation_82 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_82[0][0]'] 
                                                                                                  
 activation_83 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_83[0][0]'] 
          

                                                                                                  
 activation_91 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_91[0][0]'] 
                                                                                                  
 activation_92 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_92[0][0]'] 
                                                                                                  
 batch_normalization_93 (BatchN  (None, 5, 5, 192)   576         ['conv2d_93[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_85 (Activation)     (None, 5, 5, 320)    0           ['batch_normalization_85[0][0]'] 
                                                                                                  
 mixed9_1 

In [5]:
len(model.layers)

318

In [6]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [7]:
# Image preprocessing and data augmentation
import cv2

batch_size=8

train_datagen = ImageDataGenerator(
                         horizontal_flip = True,
                         vertical_flip = True,
                         rotation_range = 90,
                         fill_mode="constant",
                         width_shift_range = 0.2,
                         zoom_range = 0.2,
                         rescale=1./255,
                         height_shift_range = 0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train0'+vid[-1], 
                                                  target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 184 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [8]:
#Callbacks
def get_callbacks(name_weights, patience_lr, patience):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=patience, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [9]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_class_InceptionV3_v003_"+vid[-1]+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10, patience=10) # do not reduce LR here

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_steps=step_size_valid,
                    validation_data=valid_generator,
                    epochs=3,
                    verbose=1,
                    callbacks = callbacks)  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15268\2489242286.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/3
23/23 [==============================] - 23s 374ms/step - loss: 0.8303 - accuracy: 0.5707 - val_loss: 0.4518 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 2/3
23/23 [==============================] - 6s 254ms/step - loss: 0.6931 - accuracy: 0.6902 - val_loss: 0.3541 - val_accuracy: 0.8250 - lr: 0.0010
Epoch 3/3
23/23 [==============================] - 6s 258ms/step - loss: 0.5045 - accuracy: 0.7228 - val_loss: 0.3134 - val_accuracy: 0.9000 - lr: 0.0010


In [10]:
#Training stage 2
top_layer_num=round(len(model.layers)*0.75)

for layer in model.layers[:top_layer_num]:
    layer.trainable=False
for layer in model.layers[top_layer_num:]:  #last ~25% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10, patience=10) # do not reduce LR here

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_steps=step_size_valid,
                    validation_data=valid_generator,
                    epochs=10,
                    verbose=1,
                    callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15268\2189440975.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/10
23/23 [==============================] - 19s 449ms/step - loss: 0.5451 - accuracy: 0.7174 - val_loss: 0.4089 - val_accuracy: 0.8500 - lr: 1.0000e-04
Epoch 2/10
23/23 [==============================] - 3s 136ms/step - loss: 0.4824 - accuracy: 0.7880 - val_loss: 0.4957 - val_accuracy: 0.8250 - lr: 1.0000e-04
Epoch 3/10
23/23 [==============================] - 4s 150ms/step - loss: 0.5537 - accuracy: 0.7120 - val_loss: 0.4250 - val_accuracy: 0.8750 - lr: 1.0000e-04
Epoch 4/10
23/23 [==============================] - 7s 306ms/step - loss: 0.4346 - accuracy: 0.7989 - val_loss: 0.3997 - val_accuracy: 0.8500 - lr: 1.0000e-04
Epoch 5/10
23/23 [==============================] - 7s 295ms/step - loss: 0.4539 - accuracy: 0.7989 - val_loss: 0.3751 - val_accuracy: 0.8000 - lr: 1.0000e-04
Epoch 6/10
23/23 [==============================] - 7s 304ms/step - loss: 0.3944 - accuracy: 0.8424 - val_loss: 0.2998 - val_accuracy: 0.8500 - lr: 1.0000e-04
Epoch 7/10
23/23 [===========================

In [11]:
#Training stage 3
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=5, patience=5)

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_steps=step_size_valid,
                    validation_data=valid_generator,
                    epochs=100,
                    verbose=1,
                    callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15268\943335175.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/100
23/23 [==============================] - 25s 583ms/step - loss: 0.5585 - accuracy: 0.7120 - val_loss: 0.4636 - val_accuracy: 0.8000 - lr: 1.0000e-05
Epoch 2/100
23/23 [==============================] - 9s 393ms/step - loss: 0.5010 - accuracy: 0.7663 - val_loss: 0.4037 - val_accuracy: 0.8000 - lr: 1.0000e-05
Epoch 3/100
23/23 [==============================] - 9s 403ms/step - loss: 0.4488 - accuracy: 0.8207 - val_loss: 0.3157 - val_accuracy: 0.8500 - lr: 1.0000e-05
Epoch 4/100
23/23 [==============================] - 4s 150ms/step - loss: 0.4607 - accuracy: 0.7772 - val_loss: 0.3817 - val_accuracy: 0.8000 - lr: 1.0000e-05
Epoch 5/100
23/23 [==============================] - 4s 149ms/step - loss: 0.4717 - accuracy: 0.7772 - val_loss: 0.3621 - val_accuracy: 0.8000 - lr: 1.0000e-05
Epoch 6/100
23/23 [==============================] - 9s 382ms/step - loss: 0.4826 - accuracy: 0.7717 - val_loss: 0.2861 - val_accuracy: 0.8250 - lr: 1.0000e-05
Epoch 7/100
23/23 [====================

In [12]:
import os, os.path

VDIR0 = 'valid0'+vid[-1]+'/ben'
VDIR1 = 'valid0'+vid[-1]+'/mal'

valid0=len([name for name in os.listdir(VDIR0) if os.path.isfile(os.path.join(VDIR0, name))])
valid1=len([name for name in os.listdir(VDIR1) if os.path.isfile(os.path.join(VDIR1, name))])

num_of_valid_samples=valid0+valid1
# Ref:  https://stackoverflow.com/questions/2632205/how-to-count-the-number-of-files-in-a-directory-using-python

In [13]:
# Validation accuracy and validation data confusion matrix

import sklearn
valid_generator = valid_datagen.flow_from_directory('valid0'+vid[-1], 
                                                     target_size=(IMG_SIZE,IMG_SIZE),
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)

Found 44 images belonging to 2 classes.
6/6 [==============================] - 3s 141ms/step
Confusion Matrix:
[[17  5]
 [ 3 19]]
Accuracy: 0.8181818181818182
ROC AUC score: 0.9442148760330579
Precision score: 0.7916666666666666
Recall score: 0.8636363636363636
F1 score: 0.8260869565217391
MCC score: 0.6390096504226938


In [14]:
# Predicted object delete threshold = 30
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('test_upp_xcept', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)

Found 42 images belonging to 2 classes.
6/6 [==============================] - 1s 84ms/step
Confusion Matrix:
[[25  0]
 [ 9  8]]
Accuracy: 0.7857142857142857
ROC AUC score: 0.8423529411764705
Precision score: 1.0
Recall score: 0.47058823529411764
F1 score: 0.6399999999999999
MCC score: 0.5882352941176471
